In [204]:
import json 
import re
from bs4 import BeautifulSoup
import requests
DEV = True
DEBUG = False
with open("Resources/SiteInfo.json", "r") as f:
    json_dev_file = json.load(f)

def get_html(URL): #gets html in text
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
    page = requests.get(URL,headers=header) 
    soup = BeautifulSoup(page.text,'html.parser') #take html
    return soup

##going with simplicity here, 
def grab_hrefs(page_elem):
    tags = []
    tags = page_elem.find_all('a')
    links = [tag['href'] for tag in tags if 'href' in tag.attrs] #find links
    return links

def grab_element_from_id(page_elem,pg_id):
    return page_elem.find(id=pg_id)
    
def grab_scripts(page_elem,regex = re.compile('')):
    #return [script for script in page_elem.find_all("script") if regex.search(script.attrs['src'])]
    return page_elem.find_all("script")
    
#IN USE NOW 
def grab_team_hrefs(page_elem):
    links = grab_hrefs(page_elem)

    ####REWORK vvvvv to work with other states via an container
    team_links = [link for link in links if link.startswith("/ca/") and link.endswith("/basketball/") and link != "/ca/basketball/"]
    ####HARDCODED RN but can use relevant words from parse_url_context in the future to decipher relevant links
    return team_links

def grab_player_hrefs(page_elem): #eventually should make respective classes that override this function
    links = grab_hrefs(page_elem)
    relevant_pattern = r'careerid=[a-z0-9]+' #eventually, it should read off json to get a regex depending on site
    player_links = [link for link in links if re.search(relevant_pattern,link)]
    return player_links
#WIP

def grab_player_info(page_elem):
    ##there is a big script.
    player_scripts = grab_element_from_id(ex_player_pg, '__NEXT_DATA__')
    return

def get_value_from_key_list(json_file,list_of_keys):
    current_lvl = json_file
    #add error handling in future
    for key in list_of_keys:
        current_lvl = current_lvl[key]
    return current_lvl


In [205]:
##more functions?
def string_builder(str_template, dict_of_args): #use this to go to diff pages. teams -> roster -> individual player
    try:
        return str_template.format(**dict_of_args)
    except (IndexError, KeyError) as e:
        raise ValueError(f"Template/Args mismatch: {e}")

def dictionary_builder(keys,values):
    if len(keys) != len(values):
        raise ValueError(f"Mismatch: Pattern needs {len(keys)} values, got {len(values)}")
    return dict(zip(keys, values))

def extract_keys(str_template):
    key = re.compile(r'{(\w+)}')
    list_of_keys = re.findall(key,str_template)
    return list_of_keys

In [362]:
import json
import re
##IMPORT YOUR FUNCTIONS HEREEE!!!!

class Scraper_Task:
    # eventually will have async static func to drive multiple scraping processes at once
    # Class attribute to hold the global config
    structure_file = {}
    ##add dict for semaphores
    @staticmethod
    def load_structure(file_path):
        try:
            with open(file_path, "r") as f:
                Scraper_Task.structure_file = json.load(f)
        except (FileNotFoundError, json.JSONDecodeError, PermissionError) as e:
            raise RuntimeError(f"Config Load Failed: {e}")
            
    def __init__(self, site_key):
        self.seed_dict = {}
        self.scraped_data = None # To hold HTML or JSON blobs

        self.site_cfg = None
        self.scrape_cfg = None
        self.site_url = ""
        self.steps_list = []
        
        self.pre_run_validation(site_key)
        self.step_dict = {} #a list containing links for each step

        self.current_url = ""
        self.current_html = "" #RENAME TO SOUP

#this funcion here defines how a json file shold be structured 
    def pre_run_validation(self, site_key):
        cfg = Scraper_Task.structure_file.get("site_keys", {}).get(site_key)
        if not cfg:
            raise ValueError(f"Site config for '{site_key}' missing or empty in structure file.")
        scrape_cfg = cfg.get("scraping",{})
        if not scrape_cfg:
            raise ValueError(f"Scrape config for '{site_key}' missing or empty in structure file.")
        url = cfg.get("site_url")
        if not url:
            raise ValueError(f"Site url missing or null for '{site_key}'.")
        steps = scrape_cfg.get("steps", [])
        if not steps:
            raise ValueError(f"Site scrape steps missing or null for '{site_key}'.")

        self.site_key = site_key
        self.site_cfg = cfg
        self.scrape_cfg = scrape_cfg
        self.site_url = str(url)
        self.steps = steps
    
        steps = self.scrape_cfg.get("steps", [])
        
        for index, step in enumerate(steps):
            methods = step.get("scrape_method", [])
            for method in methods:
                if not (hasattr(self, method) and callable(getattr(self, method))):
                    raise AttributeError(f"Step {index} Method '{method}' is not implemented in {self.__class__.__name__}.")
            if "target" in methods and "target" not in step:
                raise KeyError(f"[{step_name}] 'target' method requested, but no 'target' configuration block was found.")

    def start_scrape(self):
        pass
    def run_step(self, step_id):
        
        pass

    #when automation occurs, we seed via each line from a markup file
    def seed(self, dictionary):
        self.seed_dict = dictionary

    #builds the entry string 
    def entry(self, step_config):
        ##seed verification
        pattern = step_config.get("href_pattern", "")
        keys = extract_keys(pattern)

        for key in keys:
            if(self.seed_dict.get(key)):
                continue
            else:
                if DEV:
                    value = input(f"DEV enabled. enter value for {key}").lower().strip()
                    self.seed_dict[key] = value
                    continue
                raise ValueError(f"Missing value for seed key: {key}")
                
        
        entry_url = self.site_url+string_builder(pattern, self.seed_dict)
        self.current_html = get_html(entry_url)
        self.current_url = entry_url

    #fill in the href pattern from seed_dict
    #on a current html, match all
    #regex is optional and within config(OR SHOULD BE)
    #retrieve all hrefs and build them
    #store in appropriate list of step within dictionary
    def match(self, step_config):
        ##match al
        ##replace with generic regex if no optional regex found

        #fill in the gaps
        match_info = step_config.get("match")
        temp_pattern = match_info.get("pattern")
        temp_key_list = extract_keys(temp_pattern)
        temp_dict = dictionary_builder(temp_key_list,[r'[^/]+' for val in temp_key_list])
        pattern = string_builder(temp_pattern,temp_dict|self.seed_dict)

        #detect optional regex to add on
        optional_regex = match_info.get("regex_pattern","")
        if optional_regex:
            pattern = f"(?=.*{optional_regex}){pattern}"

        regex = re.compile(pattern)

        #hardcode default 
        tag = match_info.get("tag","a")
        attr = match_info.get("attr","href") #change this in future for other attributes/no attributes

        matches = self.current_html.find_all(tag,**{attr: True}) #this means only find tags containing whatever the attribute is
        #filtering matches via regex

        matches = [match[attr] for match in matches if regex.search(match[attr])]
        
        i = self.steps.index(step_config)
        self.step_dict[i] = matches
        

    def target(self, step_config):
        pass

    ##here, we will need to rate limit and even more so when i introduce async
    ##Also, this is how we jump steps to perform a DFS style scraping
    def access_each(self):
        pass

    #store in json
    def store(self):
        pass

    
    def attach(self):
        pass
        
    def get_structure(self):
        return self.site_cfg
    def get_scrape_config(self):
        return self.scrape_cfg


In [363]:
##WORKS HERE
Scraper_Task.load_structure("Resources/SiteInfo.json")
scrapeCA = Scraper_Task('maxpreps')
scrapeCA.seed({"state":"ca","sport":"basketball"})
step0 = scrapeCA.get_scrape_config()['steps'][0]
step1 = scrapeCA.get_scrape_config()['steps'][1]

In [364]:
scrapeCA.current_html = current_html

In [365]:
scrapeCA.match(step1)

In [ ]:
scrapeCA.

In [329]:
current_html = scrapeCA.entry(step0)

In [ ]:
##OLD TESTING BELOW####

In [120]:
laf_ball_roster_html = get_html("https://www.maxpreps.com/ca/lafayette/acalanes-dons/basketball/roster/")

In [121]:
team_page_html = get_html("https://www.maxpreps.com/ca/basketball/schools/") 

In [122]:
team_links = grab_team_hrefs(team_page_html)
print(len(team_links)) #200 is number of teams

200


In [123]:
#FOR SOME REASON THEY HAVE A SPACE which throws off search.
#develop context based title finding 
member_links = grab_roster_hrefs(laf_ball_roster_html)
print(f'{len(member_links)}') #as of rn, there are 13 players on this team

13


In [124]:
site_url = json_dev_file['maxpreps']['site_url']
ex_player_pg = get_html(f"{site_url}{member_links[-1]}")

In [125]:
player_json_script = grab_id_parent(ex_player_pg, '__NEXT_DATA__')
json_data = json.loads(player_json_script.string)
with open('ex_player_info.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=4)


In [157]:
with open('ex_player_info.json', 'r') as f:
    ex_player_file = json.load(f)
data_map = json_dev_file['maxpreps']['data_structure']['key_to_key_maps']
data_map_keys = list(data_map.keys())
general_info_path = data_map[data_map_keys[0]]
general_info = get_value_from_key_list(ex_player_file,general_info_path)
general_info

{'careerData': {'shortCareerId': '6jneeijvodtue',
  'careerId': 'dc928233-77ba-4ead-a4ee-87e52d85d05b',
  'firstName': 'Landon',
  'lastName': 'Deily',
  'bio': None,
  'primarySport': None,
  'primaryPosition': 'PF',
  'jersey': '35',
  'heightFeet': 6,
  'heightInches': 3,
  'weight': 160,
  'graduatingClass': 2026,
  'classYear': 12,
  'facebookProfile': None,
  'twitterHandle': None,
  'hasProfileImage': True,
  'isManuallyManaged': True,
  'createdOn': '2022-11-27T03:09:00',
  'modifiedOn': '2025-11-16T01:01:38',
  'migratedOn': None,
  'migratedToCareerId': None,
  'excludeFromSearchEngines': False,
  'photoVersion': 638988516989677600,
  'photoUrl': 'https://image.maxpreps.io/careers/d/c/9/dc928233-77ba-4ead-a4ee-87e52d85d05b/thumbnail.jpg?version=638988516989677563',
  'mostRecentSchoolId': 'e6042da9-e5ce-4f18-a841-ae5e6cf48015',
  'mostRecentSportSeasonName': 'Boys Basketball Winter 25-26',
  'mostRecentSportSeasonId': 'ee7cf537-7394-4430-88ca-becaabda286c',
  'newDataAdded': 

In [164]:
ex_stats_pg = get_html("https://www.maxpreps.com/ca/lafayette/acalanes-dons/athletes/landon-deily/basketball/stats/?careerid=6jneeijvodtue")
ex_bio_pg = get_html("https://www.maxpreps.com/ca/lafayette/acalanes-dons/athletes/landon-deily/bio/?careerid=6jneeijvodtue")

In [165]:

stats_NEXT_DATA = grab_id_parent(stats_pg, '__NEXT_DATA__')
json_data = json.loads(stats_NEXT_DATA.string)
with open('Resources/ex_player_info.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=4)
    
bio_NEXT_DATA = grab_id_parent(stats_pg, '__NEXT_DATA__')
json_data = json.loads(bio_NEXT_DATA.string)
with open('Resources/ex_player_bio.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=4)

In [127]:
redundant_words = {"maxpreps","http","https","www","com"} #could also be relevant words when looking for links that match page context 
URL = "https://www.maxpreps.com/ca/basketball/schools/"
url_words = [w for w in re.split(r'[/\.:]+',URL) if w.lower() not in redundant_words and len(w) > 0]
url_words

['ca', 'basketball', 'schools']

In [128]:
target_text = soup.find_all(['h1','h2','h3'])
target_text[-1].get_text()
target_text #WIP on pinpointing exact div-- for the far future

NameError: name 'soup' is not defined

In [ ]:
def parse_url_context(URL, redundant_words): # remove maxpreps and https:// etc but keep keywords like ca
    # sc, Basketball, Boys to use and find relevant themes. This way, we can remove redundant words from links within page that may point to itself
    #can use these words and additional context to generate likelihood of teams div
    #not use right now
    #chop up url
    url_words = [w for w in re.split(r'[/\.:]+',URL) if w.lower() not in redundant_words and len(w) > 0]
    return url_words
